# Algorithm 25: makeRotX

Creates a rotation matrix for rotation around the x-axis by a given angle. Used for building sidechain conformations from torsion angles.

## Algorithm Pseudocode

![makeRotX](../imgs/algorithms/makeRotX.png)

## Source Code Location
- **File**: `AF2-source-code/model/all_atom.py`
- **Usage**: Within `torsion_angles_to_frames`

## Mathematical Definition

Rotation around x-axis by angle θ:

$$R_x(\theta) = \begin{pmatrix} 1 & 0 & 0 \\ 0 & \cos\theta & -\sin\theta \\ 0 & \sin\theta & \cos\theta \end{pmatrix}$$

In [ ]:
import numpy as np

np.random.seed(42)

In [ ]:
def make_rot_x(angle):
    """
    makeRotX - Algorithm 25.
    
    Creates rotation matrix around x-axis.
    
    Args:
        angle: Rotation angle in radians (scalar or array)
    
    Returns:
        R: Rotation matrix [..., 3, 3]
    """
    angle = np.asarray(angle)
    c = np.cos(angle)
    s = np.sin(angle)
    
    # Build rotation matrix
    R = np.zeros(angle.shape + (3, 3))
    R[..., 0, 0] = 1.0
    R[..., 1, 1] = c
    R[..., 1, 2] = -s
    R[..., 2, 1] = s
    R[..., 2, 2] = c
    
    return R


def make_rot_y(angle):
    """Rotation around y-axis (for completeness)."""
    angle = np.asarray(angle)
    c, s = np.cos(angle), np.sin(angle)
    R = np.zeros(angle.shape + (3, 3))
    R[..., 0, 0] = c
    R[..., 0, 2] = s
    R[..., 1, 1] = 1.0
    R[..., 2, 0] = -s
    R[..., 2, 2] = c
    return R


def make_rot_z(angle):
    """Rotation around z-axis (for completeness)."""
    angle = np.asarray(angle)
    c, s = np.cos(angle), np.sin(angle)
    R = np.zeros(angle.shape + (3, 3))
    R[..., 0, 0] = c
    R[..., 0, 1] = -s
    R[..., 1, 0] = s
    R[..., 1, 1] = c
    R[..., 2, 2] = 1.0
    return R

In [ ]:
# Test
print("Test makeRotX")
print("="*50)

# Test 1: Identity (angle = 0)
R0 = make_rot_x(0.0)
print(f"Rot(0) = Identity:")
print(R0)
print(f"Is identity: {np.allclose(R0, np.eye(3))}")

# Test 2: 90 degrees
R90 = make_rot_x(np.pi/2)
print(f"\nRot(90°):")
print(np.round(R90, 3))

# Test 3: Verify orthogonality
theta = np.random.rand() * 2 * np.pi
R = make_rot_x(theta)
print(f"\nOrthogonality (R @ R.T = I): {np.allclose(R @ R.T, np.eye(3))}")
print(f"det(R) = 1: {np.allclose(np.linalg.det(R), 1.0)}")

In [ ]:
# Test with batch of angles
print("\nBatched rotation:")
angles = np.array([0, np.pi/4, np.pi/2, np.pi])
R_batch = make_rot_x(angles)
print(f"Input angles: {angles}")
print(f"Output shape: {R_batch.shape}")

# Verify each is orthogonal
for i, a in enumerate(angles):
    is_ortho = np.allclose(R_batch[i] @ R_batch[i].T, np.eye(3))
    print(f"  angle={a:.2f}: orthogonal={is_ortho}")

## Application in Sidechain Building

Torsion angles (chi angles) define rotations around bonds:

1. **Chi1**: Rotation around CA-CB bond
2. **Chi2**: Rotation around CB-CG bond
3. **Chi3, Chi4**: Further sidechain rotations

```python
# Build sidechain frame
R_chi1 = make_rot_x(chi1_angle)
frame_after_chi1 = compose(backbone_frame, R_chi1)
```